# 9. 全体総まとめと独自評価指標導入

## 概要
- これまでの01-08のノートブックの内容を振り返り、また5方式の結果を集計、その際データ理解に役立つと思われる独自評価指標を導入し、各方式の結果を比較した。

### 9.1 MASE の特性と今回データの適合性

#### 9.1.1 MASE の定義

MASEは予測区間のMAEを、訓練期間の季節ナイーブ、あるいはナイーブ予測とyとのMAEで割ったものである。

$$
\text{MASE} = \frac{\frac{1}{H} \sum\limits_{t=N+1}^{N+H} |y_t - \hat{y}_t|}{\frac{1}{N-m} \sum\limits_{t=m+1}^{N} |y_t - y_{t-m}|}
$$


| 記号 | 意味 |
| :--- | :--- |
| $\text{MASE}$ | Mean Absolute Scaled Error |
| $y_t$ | 時刻 $t$ における**実際の値** (Actual value) |
| $\hat{y}_t$ | 時刻 $t$ における**予測値** (Forecast value) |
| $N$ | **学習データ**の総観測数 |
| $H$ | **テストデータ** (予測期間) の観測数（$t=N+1$ から $t=N+H$ まで） |
| $m$ | **季節性周期** (Seasonal period) （例：日次データで週周期なら $m=7$）|

#### 9.1.2 MASEと今回の使用データの適合性

今回採用した　holt-winters, SARIMAX(フーリエ項使用), prophet, lightGBM, GRU  
概ね　
- 太陽光発電量予測が　MASE　1.8-2.3程度　周期1日　（30分単位データで48コマ）
- 電力需要量予測が　MASE　0.5-0.9 程度  周期1週間　(30分単位データで336コマ)
であった。

グラフの観察も合わせ、
- 太陽光では24時間後の値が、それほど急に変わらず、主として天候要因で変化すると考えられる。1月1日と7月1日では日光の全射量、日の出時間なども大きく違うが、この1日後では差はかなり限定されよう。
- 一方電力消費では1週間後の同時刻との差は、人間の活動、天候、気温、イベントなどである程度差が大きくなることが予想される。

このような前提を置き、また予測を訓練期間60日　予測期間1日で30日分予測するので、季節ナイーブ予測の誤差はやや長めの期間で平均化され、太陽光発電の方が小さくなることが予想される。

そうなると、傾向として太陽光発電量予測のほうがMASEは高めに出てしまうので、そういった状況を確認するため次のような指標を導入してみた。

#### 9.1.3 独自指標　My_Eval_Index の導入

発想としては、

「ナイーブ季節予想と各方式の予想のMAEを比較するのであるから、分母はテスト期間のナイーブ季節予想で良いのではないか？ そもそも同じ期間の同じデータの誤差を比較して割っているので、整合性もある。」

という事である。

したがって、式　テスト期間のMAE　/ テスト期間の季節性ナイーブ予想　　
* 季節ナイーブ予想作成で除外されるデータ期間は、分子のMAE計算でも除外する。

$MAE_{test}$ / $MAE_{test\_seasonaly\_naive}$ 

と言うことで、検索し相対MAE relative MAE という用語もでてきたが、これは　相対誤差 = |予測値 - 実際値| / 実際値　で誤差の相対的な大きさを測るものがあり、自分が考えたものとは違うようである。

ここから下は　この My_Eval_Index も用いて、5方式の評価結果を比較する。

### 9.2 独自指標を含めた予測結果の評価の集計

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import warnings

# 共通モジュールのインポート
from src.data_utils import load_timeseries_data
from src.evaluation_utils import evaluate_forecast_result

# 表示設定
pd.options.display.float_format = '{:.4f}'.format

# スタイル設定
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [15, 7]
plt.rcParams['font.family'] = 'Meiryo' # Windowsの場合。Macの場合は 'Hiragino Sans' など

# 警告を非表示
warnings.filterwarnings('ignore')

In [2]:
# load data
BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR.parent / "data"
target_file = DATA_DIR / "e_gen_demand.csv"

df = load_timeseries_data(target_file)

print("データ形状:", df.shape)
print("データ期間:", df.index.min(), "から", df.index.max())
df.head()

# Train期間の定義（これより前をTrainとする）
TRAIN_LENGTH = 2880  # 60 days * 48 points/day

# 学習データを切り出し
y_train_solar = df['solar_gen_mw'].iloc[:2880]
y_train_demand = df['e_demand_mw'].iloc[:2880]

print(f"Train Solar Len: {len(y_train_solar)}")
print(f"Train Demand Len: {len(y_train_demand)}")

データ形状: (39408, 2)
データ期間: 2023-01-01 00:00:00 から 2025-03-31 23:30:00
Train Solar Len: 2880
Train Demand Len: 2880


In [3]:
results = []
preds_dir = '../results/preds/'  # Pickle保存先
file_list = [f for f in os.listdir(preds_dir) if f.endswith('.pkl')]

# 季節周期の設定
seasonal_map = {'solar': 48, 'demand': 336}
train_map = {'solar': y_train_solar, 'demand': y_train_demand}

for filename in file_list:
    # ファイル名ルール: "solar_lightgbm.pkl" → target="solar", model="lightgbm"
    # 必要に応じてsplitのロジックを調整してください
    target = 'solar' if 'solar' in filename else 'demand'
    
    # モデル名抽出（簡易的な処理）
    model_name = filename.replace('.pkl', '').replace(f'{target}_', '')
    
    # Pickle読み込み
    pred_df = pd.read_pickle(os.path.join(preds_dir, filename))
    
    # 評価実行
    metrics = evaluate_forecast_result(
        pred_df=pred_df,
        y_train=train_map[target],
        seasonal_period=seasonal_map[target]
    )
    
    # 結果格納
    res_dict = metrics.to_dict()
    res_dict['Model'] = model_name
    res_dict['Target'] = target
    results.append(res_dict)

# データフレーム化
df_results = pd.DataFrame(results)

In [4]:
# 見やすく整形：行をモデル、列をターゲット×指標に
pivot_df = df_results.pivot(index='Model', columns='Target', 
values=['MAE','RMSE', 'MAE(adjusted)', 'MAE_Naive(Test)', 'My_Eval_Index', 
        'MAE_Naive(Train)', 'MASE (Train)'])

# カラムの順序を入れ替えて見やすくする（SolarとDemandをまとめる）
pivot_df = pivot_df.swaplevel(0, 1, axis=1).sort_index(axis=1)

print("=== モデル評価結果一覧 ===")
display(pivot_df)

# CSVに保存
pivot_df.to_csv('../results/final_model_comparison_summary.csv')
print("Saved comparison to ../results/final_model_comparison_summary.csv")

=== モデル評価結果一覧 ===


Target      demand                                                 \
               MAE MAE(adjusted) MAE_Naive(Test) MAE_Naive(Train)   
Model                                                               
GRU      2119.5621     1986.4520       1911.2595        3200.9591   
SARIMAX  1712.4219     1695.4076       1908.2428        3200.9591   
hw       1663.8908     1664.6051       1908.2428        3200.9591   
lightGBM 2657.4638     2660.7534       1911.2595        3200.9591   
prophet  2163.3559     2193.3998       1908.2428        3200.9591   

Target                                            solar                \
         MASE (Train) My_Eval_Index      RMSE       MAE MAE(adjusted)   
Model                                                                   
GRU            0.6622        1.0393 2908.0458 1340.2721     1352.8759   
SARIMAX        0.5350        0.8885 2372.9847 1395.8209     1405.8266   
hw             0.5198        0.8723 2212.8888 1663.8680     1680.0904   
lightGBM       0.8302        1.3921 3356.7358 1358.2313     1349.7915   
prophet        0.6758        1.1494 2753.3141 1730.1959     1729.4279   

Target                                                                          
         MAE_Naive(Test) MAE_Naive(Train) MASE (Train) My_Eval_Index      RMSE  
Model                                                                           
GRU            1227.9985         721.5784       1.8574        1.1017 2316.5246  
SARIMAX        1228.4986         721.5784       1.9344        1.1443 2278.5357  
hw             1228.4986         721.5784       2.3059        1.3676 2740.7090  
lightGBM       1227.9985         721.5784       1.8823        1.0992 2728.5854  
prophet        1228.4986         721.5784       2.3978        1.4078 2446.8022

Saved comparison to ../results/final_model_comparison_summary.csv


上の表で左列から
```
MAE: 通常の予測値のMAE,
MAE(adjusted): 季節ナイーブ予想の期間と対応した予測MAE,
MAE_Naive(Test): テスト区間の季節ナイーブ予測のMAE,
My_Eval_Index: MAE(adjusted) / MAE_Naive(Test),
MAE_Naive(Train): Train期間の季節ナイーブ予想のMAE,
MASE (Train): 通常のMASE　MAE / MAE_Naive(Train),
RMSE: 通常のRMSE
```
となっている。

### 9.3 評価指標の考察

- 予想通り、train期間の季節ナイーブ予想のMAE:MAE_Naive(Train) は、太陽光発電(solar)が 721.6 と小さく、電力需要(demand) では3201.0と大きい。
- この結果通常MASEでは、太陽光が分母が小さいので大きめの数値、電力需要は分母が大きいので1から1を少し下回る値となっている。
- 一方　テスト期間で対象期間を揃えたMy_Eval_Index ではどちらも1前後で、「季節ナイーブ予想に対して、同じ期間でどの程度優れ、あるいは劣っているかが分かりやすくなってる。

### 9.4 予測方式の相対評価

- ただし、データの特徴はMy_Eval_Index の使用で明らかになったものの、5種類の予測方式の相対比較はどの指標でも可能である。
- レンジが太陽光発電では 0-15000mw, 電力需要=電力消費は　20000-56000程度で後者が大きく、誤差は概ね同レンジにあるので、太陽光のほうが誤差が大きいことは前提として指摘できる。
- その上で太陽光発電量予測では　lightGBM, GRUが（ただしlightGBMのRMSEは大きいので大きな誤差が発生する傾向にはある）、電力需要量予測ではholt-winters, SARIMAX (こちらは両方RMSEも低め)が優れている結果となった。

### 9.5 特徴量選択と5方式の課題

#### 9.5.1 特徴量選択

特徴量選択では、電力需要量がLASSOの正規化係数が上がるに連れ、選択する特徴量が減り誤差が大きくなるという通常の関係が見られなかった。これに対してマニュアルで正規化係数を決めたが、ランダムフォレストやlightGBM のfeature_importance を使用するなど別の手段での特徴量選択も考えられる。

#### 9.5.2 holt-winters

複数周期に対応できず限界があると思ったが、週次と日次以上の周期を持つと予想される電力需要量で良い結果を出した。これ以上の性能向上は難しい点がある。
- 太陽光発電で全てに共通するが、夜間ゼロの期間を全く除外してしまうこと。
- 予測方法で　加法、乗法どちらを用いるか、damped_trend トレンドの減衰を考慮に入れるかなどの選択
は探求の余地がある。

#### 9.5.3 SARIMAX

当初SARIMA　としてフーリエ項の外生変数を用いず、予測パラメータ推定を行って、10時間超えなど長時間を要した。フーリエ項の導入でこの点は改良され、比較的良い結果が得られるようになった。
- 改良の可能性としては、年周期（太陽光は太陽の角度、日の出日の入り時間など、消費では気温他の天候、季節イベントなどで影響は小さくないであろう）の導入が考えられる。

#### 9.5.4 Prophet

外生変数として、祝日や休日を取り込むことは想定されており、その他一般的な外生変数も使用できる。
- ます祝日を外生変数として入れ、各種設定や多岐にわたるパラメータを探索することで性能向上が期待できる。パラメータなどについては　05_Prophet_Forecast.ipynb　の末尾に記載した。

#### 9.5.5 LightGBM

- 非線形なので、太陽光の夜間ゼロなどをうまく切り分けると思ったが、大幅な他への優位は見られなかった。一方複雜な周期性を持ちそうな電力需要は苦手のようで成績は振るわなかった。
- パラメーターチューニングがまりうまく行っていなく、チューニング後小幅ながら性能低下が起こった。これに関しては方式などの見直し、採用パラメーターの拡大などが考えられる。

#### 9.5.6 GRU

- パラメータチューニングでシンプルな層の数自体を組み込んでいて、複雜な構成のモデルになることは回避している。こちらはチューニングにより性能向上を達成できており、モデルの層を増やすなど複雜化すると、予測時間が長くなるなどトレードオフの関係があり、一般的な共通の改善策を取ることが考えられる。

## 9.6　総評

- 太陽光発電量は、ある程度まで周期性を捉えたらば、夜間ゼロを除外すると性能向上は見込める。
- また年間で、太陽光の全射量に直接響く太陽光の角度、日の出日の入り時間は精度を向上させる可能性はたかいが、それ以上は　「明日晴れるか、雨や曇りなのが？」が24時間後予想では大きな要因として残りそうである。
- つまり「よく当たる天気予報情報を入手すれば良い」ということになってしまい、それ以上はリアルタイムに準じる、数時間遅れの予想（今日は晴れたのでこのパターンで発電は増加しそう等）なども考慮する必要はあるだろう。

- 電力需要予測は太陽光と異なり、経済活動など相当多くの細かいファクターの合計であろうから、こちらはこちらで難易度は高い。
- 冷暖房需要は気温の予想さえできればと、太陽光と同じ要素がある。その他はイベント、経済活動でイベントはある程度事前に予測できるものもあるが、コロナ禍のような予測が難しいものもある。
- こちらは夜間ゼロを除外するなどの方法はないので、各モデルの改善策を実施することが主な性能向上の手段となろう。